## Lendo a primeira versão via Python

In [0]:
%python
df = spark.read \
.format("delta") \
.option("versionAsOf", "1") \
.load("/user/hive/warehouse/compras")
df.show()

+---+----------+--------+---------------+----+-----+
| id|date_order|customer|        product|unit|price|
+---+----------+--------+---------------+----+-----+
|  2|2021-03-22|     Ana|AR-CONDICIONADO|   6|121.6|
|  5|2021-05-23|  Tereza|       FRIGOBAR|   3|412.0|
|  7|2021-07-25|   Sofia|        CADEIRA|   1|342.3|
|  3|2021-04-21|   Sofia|        FREEZER|   7|415.4|
|  6|2021-06-25|   Carla|           MESA|   1|124.0|
|  4|2021-04-23|  Sandra|             TV|   8|313.0|
|  1|2021-01-23|  Carlos|             TV|   5|238.0|
+---+----------+--------+---------------+----+-----+



## Contando a quantidade de registros na terceira versão via SQL

In [0]:
%sql
SELECT count(*) FROM compras VERSION AS OF 3

count(1)
6


## Contando a quantidade de registros na terceira versão via SQL - Outra forma de realizar a tarefa

In [0]:
%sql
SELECT count(*) FROM compras@v3

count(1)
6


## Contando a quantidade de registros na terceira versão via SQL - Outra forma de realizar a tarefa

In [0]:
%sql
SELECT * FROM delta.`/user/hive/warehouse/compras@v3`

id,date_order,customer,product,unit,price
2,2021-03-22,Ana,AR-CONDICIONADO,6,121.6
4,2021-04-23,Sandra,Geladeira,8,313.0
5,2021-05-23,Tereza,FRIGOBAR,3,412.0
7,2021-07-25,Sofia,CADEIRA,1,342.3
3,2021-04-21,Sofia,FREEZER,7,415.4
6,2021-06-25,Carla,MESA,1,124.0


## Descrevendo o histórico dos dados para verificar a quantidade de versões

In [0]:
%sql
DESCRIBE HISTORY '/user/hive/warehouse/compras'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2022-11-14T12:25:02.000+0000,6031105194152143,patrick_diorio@hotmail.com,RESTORE,"Map(version -> 2, timestamp -> null)",null,List(1102759827964413),1114-104148-clvtqm98,3,Serializable,false,"Map(numRestoredFiles -> 1, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 1528, numOfFilesAfterRestore -> 7, tableSizeAfterRestore -> 10926)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-11-14T10:51:40.000+0000,6031105194152143,patrick_diorio@hotmail.com,DELETE,"Map(predicate -> [""(CAST(spark_catalog.default.compras.id AS INT) = 1)""])",null,List(3189613213840133),1114-104148-clvtqm98,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2899, numDeletedRows -> 1, scanTimeMs -> 2075, numAddedFiles -> 0, rewriteTimeMs -> 823)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-11-14T10:51:29.000+0000,6031105194152143,patrick_diorio@hotmail.com,UPDATE,Map(predicate -> (cast(id#1505 as int) = 4)),null,List(3189613213840133),1114-104148-clvtqm98,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 5468, scanTimeMs -> 284, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 5161)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-11-14T10:48:36.000+0000,6031105194152143,patrick_diorio@hotmail.com,MERGE,"Map(predicate -> (CAST(spark_catalog.default.compras.id AS BIGINT) = cmp_view.id), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3189613213840133),1114-104148-clvtqm98,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 7, executionTimeMs -> 11011, numTargetRowsInserted -> 7, scanTimeMs -> 2583, numTargetRowsUpdated -> 0, numOutputRows -> 7, numTargetChangeFilesAdded -> 0, numSourceRows -> 7, numTargetFilesRemoved -> 0, rewriteTimeMs -> 8020)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-11-14T10:45:53.000+0000,6031105194152143,patrick_diorio@hotmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [""date_order""], properties -> {})",null,List(3189613213840133),1114-104148-clvtqm98,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


## Vamos reinserir o registro com ID=1 que eliminamos, uma forma de realizar o Delta Time Travel

In [0]:
%sql
INSERT INTO compras
SELECT * FROM compras VERSION AS OF 1
WHERE Id = 1

num_affected_rows,num_inserted_rows
1,1


## Exibindo os dados atualizados, após retorno da versão 1, ou seja o registro deletado foi restaurado

In [0]:
%sql
SELECT * FROM compras

id,date_order,customer,product,unit,price
2,2021-03-22,Ana,AR-CONDICIONADO,6,121.6
4,2021-04-23,Sandra,Geladeira,8,313.0
5,2021-05-23,Tereza,FRIGOBAR,3,412.0
7,2021-07-25,Sofia,CADEIRA,1,342.3
3,2021-04-21,Sofia,FREEZER,7,415.4
6,2021-06-25,Carla,MESA,1,124.0
1,2021-01-23,Carlos,TV,5,238.0
1,2021-01-23,Carlos,TV,5,238.0


## Mostrando as versões agora, depois do insert

In [0]:
%sql
DESCRIBE HISTORY '/user/hive/warehouse/compras'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2022-11-14T13:34:31.000+0000,6031105194152143,patrick_diorio@hotmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1102759827964434),1114-104148-clvtqm98,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1528)",null,Databricks-Runtime/10.4.x-scala2.12
4,2022-11-14T12:25:02.000+0000,6031105194152143,patrick_diorio@hotmail.com,RESTORE,"Map(version -> 2, timestamp -> null)",null,List(1102759827964413),1114-104148-clvtqm98,3,Serializable,false,"Map(numRestoredFiles -> 1, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 1528, numOfFilesAfterRestore -> 7, tableSizeAfterRestore -> 10926)",null,Databricks-Runtime/10.4.x-scala2.12
3,2022-11-14T10:51:40.000+0000,6031105194152143,patrick_diorio@hotmail.com,DELETE,"Map(predicate -> [""(CAST(spark_catalog.default.compras.id AS INT) = 1)""])",null,List(3189613213840133),1114-104148-clvtqm98,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2899, numDeletedRows -> 1, scanTimeMs -> 2075, numAddedFiles -> 0, rewriteTimeMs -> 823)",null,Databricks-Runtime/10.4.x-scala2.12
2,2022-11-14T10:51:29.000+0000,6031105194152143,patrick_diorio@hotmail.com,UPDATE,Map(predicate -> (cast(id#1505 as int) = 4)),null,List(3189613213840133),1114-104148-clvtqm98,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 5468, scanTimeMs -> 284, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 5161)",null,Databricks-Runtime/10.4.x-scala2.12
1,2022-11-14T10:48:36.000+0000,6031105194152143,patrick_diorio@hotmail.com,MERGE,"Map(predicate -> (CAST(spark_catalog.default.compras.id AS BIGINT) = cmp_view.id), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3189613213840133),1114-104148-clvtqm98,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 7, executionTimeMs -> 11011, numTargetRowsInserted -> 7, scanTimeMs -> 2583, numTargetRowsUpdated -> 0, numOutputRows -> 7, numTargetChangeFilesAdded -> 0, numSourceRows -> 7, numTargetFilesRemoved -> 0, rewriteTimeMs -> 8020)",null,Databricks-Runtime/10.4.x-scala2.12
0,2022-11-14T10:45:53.000+0000,6031105194152143,patrick_diorio@hotmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [""date_order""], properties -> {})",null,List(3189613213840133),1114-104148-clvtqm98,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


## Verificando quantos registro é a diferença da versão atual, para a versão 3

In [0]:
%sql
SELECT count(distinct ID) - (SELECT count(distinct ID) FROM compras VERSION AS OF 3) as `Diferença de registros`
FROM compras

Diferença de registros
1
